In [2]:
import requests
import pandas as pd
import numpy as np
import folium
import lxml
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [5]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))
        
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [7]:
toronto_df.shape

(210, 3)

In [8]:
toronto_df = toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
toronto_df.loc[toronto_df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


In [10]:
toronto_df['Neighborhood'].replace(['Not assigned'], "Queen's Park")

0                                         Rouge, Malvern
1                 Highland Creek, Rouge Hill, Port Union
2                      Guildwood, Morningside, West Hill
3                                                 Woburn
4                                              Cedarbrae
5                                    Scarborough Village
6            East Birchmount Park, Ionview, Kennedy Park
7                        Clairlea, Golden Mile, Oakridge
8        Cliffcrest, Cliffside, Scarborough Village West
9                            Birch Cliff, Cliffside West
10     Dorset Park, Scarborough Town Centre, Wexford ...
11                                     Maryvale, Wexford
12                                             Agincourt
13               Clarks Corners, Sullivan, Tam O'Shanter
14     Agincourt North, L'Amoreaux East, Milliken, St...
15                                       L'Amoreaux West
16                                           Upper Rouge
17                             

In [11]:
toronto_df.loc[toronto_df['Borough'] == "Queen's Park"]

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


In [12]:
toronto_df.shape

(103, 3)

In [13]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
coordinates.columns = ['PostalCode', 'Latitude','Longitude']
toronto_df_with_Coords = toronto_df.merge(coordinates, on="PostalCode", how="left")
toronto_df_with_Coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_with_Coords['Borough'].unique()),
        toronto_df_with_Coords.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [19]:
toronto_df_with_Coords.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           11
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [20]:
boroughs = toronto_df_with_Coords['Borough'].unique().tolist()

In [21]:
boroughs

['Scarborough',
 'North York',
 'East York',
 'East Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'York',
 'West Toronto',
 'Mississauga',
 'Etobicoke',
 "Queen's Park"]

In [23]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [27]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))


In [32]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_with_Coords['Latitude'], toronto_df_with_Coords['Longitude'], toronto_df_with_Coords['Borough'], toronto_df_with_Coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill=True,
        fill_color=borough_color[borough],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [33]:
CLIENT_ID = 'OYKQTSLK4CCFCUNN0XNJXHXIO5GPMGVWQVHOPSDTHW44C1J3' # your Foursquare ID
CLIENT_SECRET = 'XJA1WJKV0QIKERAU3PSHE0AYPYRFO33PPAIJAGPAX02NHBMB'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=toronto_df_with_Coords['Neighborhood'],
                                   latitudes=toronto_df_with_Coords['Latitude'],
                                   longitudes=toronto_df_with_Coords['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [38]:
toronto_venues.shape

(2236, 7)

In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Downsview North, Wilson Heights",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


In [41]:
toronto_venues['Venue Category'].unique()[:100]

array(['Fast Food Restaurant', 'Bar', 'History Museum', 'Pizza Place',
       'Electronics Store', 'Spa', 'Mexican Restaurant',
       'Rental Car Location', 'Medical Center', 'Intersection',
       'Breakfast Spot', 'Coffee Shop', 'Korean Restaurant',
       'Hakka Restaurant', 'Caribbean Restaurant', 'Thai Restaurant',
       'Athletics & Sports', 'Bank', 'Gas Station', 'Bakery',
       'Fried Chicken Joint', 'Playground', 'Department Store',
       'Discount Store', 'Hobby Shop', 'Bus Station', 'Ice Cream Shop',
       'Bus Line', 'Park', 'Soccer Field', 'Motel', 'Movie Theater',
       'American Restaurant', 'Café', 'General Entertainment',
       'Skating Rink', 'College Stadium', 'Indian Restaurant',
       'Chinese Restaurant', 'Pet Store', 'Vietnamese Restaurant',
       'Brewery', 'Sandwich Place', 'Middle Eastern Restaurant',
       'Shopping Mall', 'Auto Garage', 'Lounge',
       'Latin American Restaurant', 'Italian Restaurant', 'Noodle House',
       'Pharmacy', 'Grocery S

In [54]:
"Gym" in toronto_venues['Venue Category'].unique()

True

In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,0.0,0.000000,0.00
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.017857,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00


In [55]:
len(toronto_grouped[toronto_grouped["Gym"] > 0])

22

In [57]:
gym = toronto_grouped[["Neighborhood","Gym"]]

In [58]:
gym.head()

,Neighborhood,Gym
0,"Adelaide, King, Richmond",0.020000
1,Agincourt,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000
4,"Alderwood, Long Branch",0.090909


In [106]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 0, 0, 1, 1, 1, 1, 1])

In [107]:
to_merged = gym.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [108]:
to_merged.head()

,Neighborhood,Gym,Cluster Labels
0,"Adelaide, King, Richmond",0.020000,1
1,Agincourt,0.000000,1
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,2
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0
4,"Alderwood, Long Branch",0.090909,0


In [109]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
to_merged.head()

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.02,1,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
0,"Adelaide, King, Richmond",0.02,1,43.650571,-79.384568,The Keg Steakhouse + Bar,43.649937,-79.384196,Steakhouse
0,"Adelaide, King, Richmond",0.02,1,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.02,1,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
0,"Adelaide, King, Richmond",0.02,1,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to the map
markers_colors = {}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'

for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [111]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,"Alderwood, Long Branch",0.090909,0,43.602414,-79.543484,Toronto Gymnastics International,43.599832,-79.542924,Gym
17,Canada Post Gateway Processing Centre,0.083333,0,43.636966,-79.615819,LA Fitness,43.637794,-79.621765,Gym
73,Queen's Park,0.052632,0,43.662301,-79.389494,YMCA,43.662753,-79.384849,Gym
73,Queen's Park,0.052632,0,43.662301,-79.389494,Hart House Gym,43.664172,-79.394888,Gym


In [112]:
#Cluster 2
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.020000,1,43.650571,-79.384568,Equinox Bay Street,43.648100,-79.379989,Gym
0,"Adelaide, King, Richmond",0.020000,1,43.650571,-79.384568,GoodLife Fitness Toronto Bell Trinity Centre,43.653436,-79.382314,Gym
11,"Brockton, Exhibition Place, Parkdale Village",0.045455,1,43.636847,-79.428191,Reebok Crossfit Liberty Village,43.637036,-79.424802,Gym
22,Church and Wellesley,0.024096,1,43.665860,-79.383160,Hone Fitness,43.667484,-79.385510,Gym
22,Church and Wellesley,0.024096,1,43.665860,-79.383160,YMCA,43.662753,-79.384849,Gym
27,"Commerce Court, Victoria Hotel",0.040000,1,43.648198,-79.379817,Equinox Bay Street,43.648100,-79.379989,Gym
27,"Commerce Court, Victoria Hotel",0.040000,1,43.648198,-79.379817,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
27,"Commerce Court, Victoria Hotel",0.040000,1,43.648198,-79.379817,Toronto Athletic Club,43.647360,-79.381177,Gym
27,"Commerce Court, Victoria Hotel",0.040000,1,43.648198,-79.379817,GoodLife Fitness Toronto Street Women's Only,43.650400,-79.376700,Gym
28,Davisville,0.055556,1,43.704324,-79.388790,GoodLife Fitness,43.700802,-79.386417,Gym


In [113]:
#Cluster 3
to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [114]:
#Cluster 4
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [115]:
#Cluster 5
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


### CONCLUSION

Many gyms are in areas that belong to first two clusters. So it would be wise to think as an opportunity to search for proper areas that belong to cluster 3,4,5. Specifically, if we search the map, area near Rathburn road and Anglesey boulevard seems a nice spot.